# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
con = sqlite3.Connection('data.sqlite')
pd.read_sql("SELECT name FROM sqlite_master WHERE type ='table'",con)

## Write an Equivalent Query using a Subquery

```SQL
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';
```

In [ ]:
pd.read_sql("""select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       join orders 
       using(customerNumber)
       where orderDate = '2003-01-31';""",con)

In [ ]:
pd.read_sql("""
select customerNumber,
       contactLastName,
       contactFirstName
       from customers
       where customerNumber IN (select customerNumber
           FROM orders
           where orderDate = '2003-01-31')""",con)

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [ ]:
pd.read_sql("""
SELECT productName, count(orderNumber) numOrders, sum(quantityOrdered) quantitySold
FROM products
JOIN orderdetails USING(productCode)
GROUP BY productCode
ORDER BY sum(quantityOrdered)
""",con)

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

In [ ]:
pd.read_sql("""
SELECT productCode, count(peopleOrdered) peopleCount
FROM (SELECT DISTINCT productCode, customerNumber peopleOrdered
        FROM products
        JOIN orderDetails USING (productCode)
        JOIN orders USING (orderNumber)
        JOIN customers USING (customerNumber)
)
GROUP BY productCode
HAVING peopleCount < 20
ORDER BY peopleCount DESC
""",con)

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [ ]:
#Your code here:
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [ ]:
pd.read_sql("""
SELECT DISTINCT employeeNumber, e.firstName, e.lastName, o.City, officeCode
FROM offices o
JOIN employees e USING (officeCode)
JOIN customers ON employeeNumber = salesRepEmployeeNumber
JOIN orders USING (customerNumber)
JOIN orderDetails USING (orderNumber)
JOIN products USING (productCode)
WHERE productCode IN (SELECT productCode
                        FROM (SELECT DISTINCT productCode, customerNumber peopleOrdered
                                FROM products
                                JOIN orderDetails USING (productCode)
                                JOIN orders USING (orderNumber)
                                JOIN customers USING (customerNumber)
                                )
                      GROUP BY productCode
                      HAVING count(peopleOrdered) < 20)

""",con)

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [ ]:
pd.read_sql("""
SELECT employeeNumber, e.firstName, e.lastName, count(customerNumber) numCustomers
FROM employees e
JOIN customers ON employeeNumber = salesRepEmployeeNumber
GROUP BY employeeNumber
HAVING avg(creditLimit) > 15
""",con)

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!